In [2]:
from datetime import datetime
import pandas as pd
import time 
import json 
import os
import selenium
import numpy as np
from time import strftime, localtime
import win32com.client as win32
import os
import time
import csv
import re 
from unidecode import unidecode

# 1. Download the .xml Files from 'https://fbref.com/en/' Website

# 2. Convert the .xml to .xlsx files and save them

In [3]:
os.chdir(r"Raw Data\Match Data")
fnames = os.listdir()

for fname in fnames:
    if fname == "xlsx_files":
        continue
    else:
        excel = win32.gencache.EnsureDispatch('Excel.Application')
        wb = excel.Workbooks.Open(os.path.join(os.getcwd(), fname))
        wb.SaveAs(os.path.join(os.getcwd(), "xlsx_files", fname) + "x", FileFormat = 51)    #FileFormat = 51 is for .xlsx extension
        wb.Close()                                                                          #FileFormat = 56 is for .xls extension
        excel.Application.Quit()

# 3. Adjusting essential criteria in the DataFrame

In [5]:
tournaments = ['Belgian First Division A', 'Bundesliga', 'Champions League',
               'Eredivisie', 'Europa League', 'La Liga', 'Ligue 1', 'Premier League', 
               'Primeira Liga', 'Seria A', 'Super Lig', 'HNL', 'Ekstraklasa', 
               'Scottish Premiership', 'Swiss Super League']

international_tournaments = ['Champions League', 'Europa League']

def establish_df(file):
    
    def filename_tournament_season(filename):
        tournament_name = " ".join([x.title() for x in filename[:-11] .replace("_", " ").split()])
        season = filename[-10:-5].replace("_", "/")
        return tournament_name, season
    
    def split_score(score_raw):
         return re.findall(r'\d+', score_raw)
    
    def adjust_home(home_team):
        return " ".join(home_team.split(" ")[:-1])
    
    def adjust_away(away_team):
        return " ".join(away_team.split(" ")[1:])
    
    df = pd.read_excel(f"{file}", 
                       usecols = ["Date", "Home", "Score", "Away"])
    
    df = df.drop(df.loc[df["Score"].isnull()].index.to_list(), axis = 0)
    
    df["Tournament"] = filename_tournament_season(file)[0]
    df["Season"] = filename_tournament_season(file)[1]
    
    # SCORE CORRECTION => Score_Home, Score_Away
    df["Score_Home"] = df["Score"].transform(lambda x: int(split_score(x)[0]))
    df["Score_Away"] = df["Score"].transform(lambda x: int(split_score(x)[1]))
    
    # Specify which team wins or if it is a draw using boolean variables.
    df["Home_Wins"] = np.select([df["Score_Home"] > df["Score_Away"], df["Score_Home"] < df["Score_Away"], df["Score_Home"] == df["Score_Away"]], [True, False, False])
    df["Away_Wins"] = np.select([df["Score_Home"] < df["Score_Away"], df["Score_Home"] > df["Score_Away"], df["Score_Home"] == df["Score_Away"]], [True, False, False])
    df["Draw"] = np.select([df["Score_Home"] == df["Score_Away"], df["Score_Home"] != df["Score_Away"]], [True, False])
    
    # Specify whether the tournament is international or not with boolean variable (0 - Domestic tournament, 1 - International Tournament) 
    df["Domestic0_International1"] = np.select([df["Tournament"].isin(international_tournaments), ~df["Tournament"].isin(international_tournaments)], [True, False])
    
    # For the international competitions, take out the country code for Home and Away Teams
    international_home = df[df.Tournament.isin(international_tournaments)]["Home"]
    international_home = international_home.apply(lambda x: adjust_home(x))
    df.update(international_home)
    international_away = df[df.Tournament.isin(international_tournaments)]["Away"]
    international_away = international_away.apply(lambda x: adjust_away(x))
    df.update(international_away)
    
    # Put columns in wanted order
    df = df[["Tournament", "Domestic0_International1", "Season", "Date", "Home", "Score_Home", "Score_Away", "Away", "Home_Wins", "Away_Wins", "Draw"]]
    return df

# concatenate all the existing ajusted DataFrames    
def concatenate_dfs(df_list):
    df_all = pd.concat([df for df in df_list], ignore_index=True)    
    return df_all
    
os.chdir(r"Raw Data\Match Data\xlsx_files")
match_files = os.listdir() 
df_list = []
for i in match_files: 
    df_list.append(establish_df(i))
    
df = concatenate_dfs(df_list)

In [6]:
df

,Tournament,Domestic0_International1,Season,Date,Home,Score_Home,Score_Away,Away,Home_Wins,Away_Wins,Draw
0,Belgian First Division A,0,18/19,2018-07-27,Standard LiÃ¨ge,3,2,Gent,1,0,0
1,Belgian First Division A,0,18/19,2018-07-28,Kortrijk,1,4,Anderlecht,0,1,0
2,Belgian First Division A,0,18/19,2018-07-28,Sint-Truiden,0,0,Cercle Brugge,0,0,1
3,Belgian First Division A,0,18/19,2018-07-28,Zulte Waregem,2,2,Waasland-Bev,0,0,1
4,Belgian First Division A,0,18/19,2018-07-28,Oostende,2,1,Excel Mouscron,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
20280,Swiss Super League,0,22/23,2023-04-29,Lugano,5,1,Grasshopper,1,0,0
20281,Swiss Super League,0,22/23,2023-04-29,Servette FC,1,1,St. Gallen,0,0,1
20282,Swiss Super League,0,22/23,2023-04-30,ZÃ¼rich,2,2,Sion,0,0,1
20283,Swiss Super League,0,22/23,2023-04-30,Young Boys,5,1,Luzern,1,0,0


# 4. Adjust the Club Names for the ones who have tokens

In [8]:
# The format of the text is not correct and needs to be adjusted. For this purpose, the teams which are expected to be analyzed will be mapped only.

""" 
teams_of_interest =  ["AC Milan", "Arsenal", "Alanyaspor", 
                      "AS Roma", "Atletico De Madrid", "Aston Villa", 
                      "FC Barcelona", "Manchester City", 
                      "Everton", "RCD Espanyol", "Fenerbahce", 
                      "Fatih Karagümrük SK", "Galatasaray", "Inter Milan", 
                      "Juventus", "S.S. Lazio", "Leeds United", 
                      "Napoli", "FC Porto", "Paris Saint-Germain", "Valencia CF"] 
                          
"""


changes_tbd = {"Milan" : "AC Milan", 
               "Roma" : "AS Roma",
               "AtlÃ©tico Madrid" : "Atletico De Madrid", 
               "Barcelona" : "FC Barcelona",
               "Espanyol" : "RCD Espanyol", 
               "FenerbahÃ§e" : "Fenerbahce", 
               "Fatih KaragÃ¼mrÃ¼k" : "Fatih Karagumruk SK", 
               "Inter" : "Inter Milan", 
               "S.S. Lazio" : "Lazio", 
               "Porto" : "FC Porto", 
               "Paris S-G" : "Paris Saint-Germain",
               "Valencia" : "Valencia CF"}

def clubname_mapper(mapping_obj):
    changes_tbd = {"Milan" : "AC Milan", 
                   "Roma" : "AS Roma",
                   "AtlÃ©tico Madrid" : "Atletico De Madrid", 
                   "Barcelona" : "FC Barcelona",
                   "Espanyol" : "RCD Espanyol", 
                   "FenerbahÃ§e" : "Fenerbahce", 
                   "Fatih KaragÃ¼mrÃ¼k" : "Fatih Karagumruk SK", 
                   "Inter" : "Inter Milan", 
                   "S.S. Lazio" : "Lazio", 
                   "Porto" : "FC Porto", 
                   "Paris S-G" : "Paris Saint-Germain",
                   "Valencia" : "Valencia CF"}
    
    if mapping_obj in changes_tbd: 
        return changes_tbd[mapping_obj]
    else: 
        return mapping_obj

    
df1 = df.copy()    
df1["Home"] = df1["Home"].apply(lambda x: clubname_mapper(x))
df1["Away"] = df1["Away"].apply(lambda x: clubname_mapper(x))
df1

,Tournament,Domestic0_International1,Season,Date,Home,Score_Home,Score_Away,Away,Home_Wins,Away_Wins,Draw
0,Belgian First Division A,0,18/19,2018-07-27,Standard LiÃ¨ge,3,2,Gent,1,0,0
1,Belgian First Division A,0,18/19,2018-07-28,Kortrijk,1,4,Anderlecht,0,1,0
2,Belgian First Division A,0,18/19,2018-07-28,Sint-Truiden,0,0,Cercle Brugge,0,0,1
3,Belgian First Division A,0,18/19,2018-07-28,Zulte Waregem,2,2,Waasland-Bev,0,0,1
4,Belgian First Division A,0,18/19,2018-07-28,Oostende,2,1,Excel Mouscron,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
20280,Swiss Super League,0,22/23,2023-04-29,Lugano,5,1,Grasshopper,1,0,0
20281,Swiss Super League,0,22/23,2023-04-29,Servette FC,1,1,St. Gallen,0,0,1
20282,Swiss Super League,0,22/23,2023-04-30,ZÃ¼rich,2,2,Sion,0,0,1
20283,Swiss Super League,0,22/23,2023-04-30,Young Boys,5,1,Luzern,1,0,0


# 5. Save the data as csv file

In [13]:
os.chdir(r"Datasets for Merging")
df1.to_csv("match_data_dataset.csv", index = False)